## NYT vs Goodreads 

- In this project, we would like to look at New York Times best seller data alongside Goodreads data. This will create a database in which books can be compared on two levels: initial attraction, considering that NYT rankings are based off sales, and reader feedback, as Goodreads ratings are provided by readers after they finish a book. 
    - This gives two different aspects regarding a book's performance: initial expectations versus how the book actually held up. 

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

## Reading the CSVs into DataFrames

In [2]:
path_NYT = "Resources/NYT.csv"
NYT_df = pd.read_csv(path_NYT)
NYT_df.head()

,published_date,list_name,list_name_encoded,rank,isbn13,isbn10,title,author,description,amazon_product_url,price,weeks_on_list
0,2010-01-03,Chapter Books,chapter-books,1,9780316036245,0316036242,WITCH AND WIZARD,James Patterson and Gabrielle Charbonnet,"One of each, brother and sister, flex their ne...",https://www.amazon.com/Witch-Wizard-James-Patt...,17.99,1
1,2010-01-03,Chapter Books,chapter-books,2,9780439023481,0439023483,THE HUNGER GAMES,Suzanne Collins,"In a dystopian future, a girl fights for survi...",https://www.amazon.com/The-Hunger-Games-Suzann...,17.99,67
2,2010-01-03,Chapter Books,chapter-books,3,9780439023498,0439023491,CATCHING FIRE,Suzanne Collins,"The protagonist of ""The Hunger Games"" returns.",https://www.amazon.com/Catching-Fire-Hunger-Ga...,17.99,16
3,2010-01-03,Chapter Books,chapter-books,4,9780763644109,0763644102,THE MAGICIAN’S ELEPHANT,Kate DiCamillo,An orphan in search of his sister follows a fo...,https://www.amazon.com/The-Magicians-Elephant-...,16.99,15
4,2010-01-03,Chapter Books,chapter-books,5,9780385738934,0385738935,FALLEN,Lauren Kate,"Thwarted love among misfits at a Savannah, Ga....",https://www.amazon.com/Fallen-Lauren-Kate/dp/0...,17.99,2


In [3]:
Goodreads = "Resources/Goodreads_all.csv"
Goodreads_df = pd.read_csv(Goodreads, sep=",")
Goodreads_df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


## Cleaning up the DataFrames

In [4]:
# cleaning up the NYT dataframe
NYT_cols = ['rank', 'isbn13', 'title', 'author', 'weeks_on_list']
NYT_clean = NYT_df[NYT_cols].copy()

In [5]:
NYT_clean = NYT_df[['rank', 'isbn13', 'title', 'author', 'weeks_on_list']].drop_duplicates()
NYT_clean.head()

,rank,isbn13,title,author,weeks_on_list
0,1,9780316036245,WITCH AND WIZARD,James Patterson and Gabrielle Charbonnet,1
1,2,9780439023481,THE HUNGER GAMES,Suzanne Collins,67
2,3,9780439023498,CATCHING FIRE,Suzanne Collins,16
3,4,9780763644109,THE MAGICIAN’S ELEPHANT,Kate DiCamillo,15
4,5,9780385738934,FALLEN,Lauren Kate,2


In [6]:
#cleaning up the Goodreads dataframe
goodreads_cols = ['bookID', 'title', 'authors', 'average_rating', 'isbn13']
goodreads_clean = Goodreads_df[goodreads_cols].copy()

In [7]:
goodreads_clean = Goodreads_df[['bookID', 'title', 'authors', 'average_rating', 'isbn13']].drop_duplicates()
goodreads_clean.set_index('bookID', inplace=True)
goodreads_clean.head()

,title,authors,average_rating,isbn13
bookID,,,,
1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,9780439785969
2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,9780439358071
4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,9780439554893
5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,9780439655484
8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,9780439682589


## Database Connection

In [8]:
from config import db_user, db_pwd, db_host, db_port, db_name 

In [9]:
connection_string = f"{db_user}:{db_pwd}@{db_host}:{db_port}/"
engine = create_engine(f'postgresql://{connection_string}')

try:
    engine.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {db_name}")
except Exception as e:
    print(e)  

(psycopg2.errors.DuplicateDatabase) database "books" already exists

[SQL: CREATE DATABASE books]
(Background on this error at: http://sqlalche.me/e/13/f405)


In [10]:
connection_string = f"{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}"

engine = create_engine(f'postgresql://{connection_string}')

In [11]:
engine.table_names()

[]

## Dataframes to Databases

In [12]:
NYT_clean.to_sql(name='nyt', con=engine, if_exists='replace', index=False)

In [13]:
goodreads_clean.to_sql(name='goodreads', con=engine, if_exists='replace', index=True)

In [14]:
engine.table_names()

['nyt', 'goodreads']

In [15]:
pd.read_sql('Select * from nyt', engine)

,rank,isbn13,title,author,weeks_on_list
0,1,9780316036245,WITCH AND WIZARD,James Patterson and Gabrielle Charbonnet,1
1,2,9780439023481,THE HUNGER GAMES,Suzanne Collins,67
2,3,9780439023498,CATCHING FIRE,Suzanne Collins,16
3,4,9780763644109,THE MAGICIAN’S ELEPHANT,Kate DiCamillo,15
4,5,9780385738934,FALLEN,Lauren Kate,2
...,...,...,...,...,...
42246,1,9781250170996,CHILDREN OF VIRTUE AND VENGEANCE,Tomi Adeyemi,2
42247,2,9781524714680,ONE OF US IS LYING,Karen M McManus,99
42248,3,9781250170972,CHILDREN OF BLOOD AND BONE,Tomi Adeyemi,93
42249,4,9781534451568,FIVE FEET APART,Rachael Lippincott with Mikki Daughtry and Tob...,54


# Report

## Extract
- Our original data was from Kaggle; we found a CSVs with data on the New York Times BestSellers List from 2009 to 2019 and data on Goodreads books for the same time perioid. 
## Transform 
- Mainly, we dropped extraneous columns that were not very relevant for the type of database we wanted to create, as well as dropping any duplicate records. 
## Load
- We loaded the final two tables, one with the New York Times data, and one with the Goodreads Data, into a 'books' database in pgAdmin. 